In [11]:
import os
import csv
import pandas as pd

In [12]:
hmda = pd.read_csv('../input_data/state_AZ-CA.csv')

C:\Users\twsca\AppData\Local\Temp\ipykernel_23224\1184079942.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  hmda = pd.read_csv('../input_data/state_AZ-CA.csv')


In [13]:
hmda['denial'] = hmda['action_taken'] == 3

In [14]:
def hmda_bank(row):
  if row['lei'] == 'QOT5WN9RBKQTFRVKEV31':
    hmda_bank = 'Bank of West'
  elif row['lei'] == '3Y4U8VZURTYWI1W2K376':
    hmda_bank = 'BMO Harris'
  else: 
    hmda_bank = 'All Other Banks'
  return hmda_bank

In [15]:
hmda['which_bank'] = hmda.apply(hmda_bank,axis=1)
hmda['which_bank'].value_counts()

All Other Banks    4179143
Bank of West         19432
BMO Harris            3378
Name: which_bank, dtype: int64

In [16]:
hmda.groupby(['which_bank', 'loan_type', 'loan_purpose'])['denial'].mean()

which_bank       loan_type  loan_purpose
All Other Banks  1          1               0.065778
                            2               0.304567
                            4               0.296351
                            5               0.012958
                            31              0.086099
                            32              0.091971
                 2          1               0.066455
                            2               0.117647
                            4               0.131186
                            5               0.000000
                            31              0.115967
                            32              0.182224
                 3          1               0.051772
                            2               0.150993
                            4               0.589958
                            5               0.000000
                            31              0.079526
                            32              0.138457
     

In [17]:
# Create the pivot table
pivot = hmda.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='denial', aggfunc='mean')

# The pivot table shows the mean denial rate for each combination of bank, loan type, and loan purpose
print(pivot)


which_bank              All Other Banks  BMO Harris  Bank of West
loan_type loan_purpose                                           
1         1                    0.065778    0.107143      0.108129
          2                    0.304567    0.357143      0.353585
          4                    0.296351    0.338645      0.408051
          5                    0.012958         NaN      0.000000
          31                   0.086099    0.086207      0.270278
          32                   0.091971    0.227687      0.197051
2         1                    0.066455    0.500000      0.272727
          2                    0.117647         NaN      1.000000
          4                    0.131186         NaN      1.000000
          5                    0.000000         NaN      0.000000
          31                   0.115967         NaN      0.545455
          32                   0.182224         NaN      0.536585
3         1                    0.051772         NaN      0.000000
          

In [20]:
pivot['diff_BMO'] = pivot['BMO Harris'] - pivot['All Other Banks']
pivot['diff_BOW'] = pivot['Bank of West'] - pivot['All Other Banks']
pivot.style.format('{:,.2f}')
print(pivot)

which_bank              All Other Banks  BMO Harris  Bank of West  diff_BMO  \
loan_type loan_purpose                                                        
1         1                    0.065778    0.107143      0.108129  0.041364   
          2                    0.304567    0.357143      0.353585  0.052576   
          4                    0.296351    0.338645      0.408051  0.042295   
          5                    0.012958         NaN      0.000000       NaN   
          31                   0.086099    0.086207      0.270278  0.000107   
          32                   0.091971    0.227687      0.197051  0.135715   
2         1                    0.066455    0.500000      0.272727  0.433545   
          2                    0.117647         NaN      1.000000       NaN   
          4                    0.131186         NaN      1.000000       NaN   
          5                    0.000000         NaN      0.000000       NaN   
          31                   0.115967         NaN 